# Lab-03 Neural Networks

## Loading Data

In [2]:
import os
import re
import cv2

images = []
pose_name = []
images_path = []
images_pixels = []
labels = []

dict = {}

i=0
dataPath = './yogaData'

for directory in os.listdir(dataPath):
    dirPath = os.path.join(dataPath, directory)
    if os.path.isdir(dirPath):
        pose_name.append(directory)
        for img in os.listdir(dirPath):
            if len(re.findall('.png',img.lower())) != 0 or len(re.findall('.jpg',img.lower())) != 0 or len(re.findall('.jpeg',img.lower())) != 0:
                img_path = os.path.join(dirPath,img)
                images.append(img)
                images_path.append(img_path)
                img_pix = cv2.imread(img_path,1)
                images_pixels.append(cv2.resize(img_pix, (100,100)))
                labels.append(i)
        
    i = i+1
            
print("Total images: ", len(images))
print("Total images path: ", len(images_path))
print("Total postures: ", len(pose_name))
print("Total images_pixels: ", len(images_pixels))  

Total images:  2177
Total images path:  2177
Total postures:  9
Total images_pixels:  2177


In [3]:
import random

shuffled = list(zip(images_pixels,labels))
random.shuffle(shuffled)

train_data, labels_data = zip(*shuffled)

In [4]:
#reminder to switch from tensorflow.keras.utils to keras.utils
from keras.utils import to_categorical
import numpy as np

X_data = np.array(train_data)/255
Y_data =  to_categorical(labels_data, num_classes = 10)

## Split the Data

In [5]:
#Now split X_data and Y_data into training and validation splits.
from sklearn.model_selection import train_test_split 

X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=1) 

## Random Forest

Create a Random Forest Model (feel free to use XGBoost!)
b. Make sure to include the following parameters, feel free to play around with the
exact values for each:
i. criterion
ii. min_samples_size
iii. min_samples_leaf
iv. max_features
v. bootstrap
vi. random_state
c. Explain what each of these parameters do.
d. Fit the data
i. You might have to reshape X_train and X_val
1. Samples, nx, ny, nrgb = X_train.shape
2. X_train2 = X_train.reshape((nsamples, nx * ny * nrgb))
3. Do the same for X_val
e. Predict and get the accuracy_score.
f. Create a classification_report (from sklearn.metrics)


In [6]:
#working with image folder so Random Forest Classifier 

from xgboost import XGBClassifier 

model = XGBClassifier() 
#n_estimators typically increased until no further improvements is observed 

model.fit(X_train, Y_train)

ValueError: Please reshape the input data into 2-dimensional matrix.

## Convolutional Neural Network

### DataLoader

In [ ]:
from torch.utils.data import DataLoader

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_sample = self.X[idx]
        y_sample = self.Y[idx]
        
        if self.transform:
            x_sample = self.transform(x_sample)
            
        return x_sample, y_sample

In [ ]:
from torchvision import transforms

img_transform = transforms.Compose([
    transforms.ToTensor(),  # Converts the image to a PyTorch tensor
])

### Neural Network